In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
HOPSWORKS_PROJECT_NAME = "ml_prod_pipline"

In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-val pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / ".env")

HOPSWORKS_API_KEY = os.environ["HOPSWORKS_API_KEY"]

In [4]:
from datetime import datetime
import pandas as pd
from src.data import loadRawData

from_year = 2022
to_year = datetime.now().year

print(f"Downloading raw data from {from_year} to {to_year}")

rides = pd.DataFrame()

for year in range(from_year, to_year + 1):
    
    # Download data for the whole year
    rides_one_year = loadRawData(year)
    
    # Append Rows
    rides = pd.concat([rides, rides_one_year])
    


File for 2022_01 not found, Downloading file...
File for 2022_02 not found, Downloading file...
File for 2022_03 not found, Downloading file...
File for 2022_04 not found, Downloading file...
File for 2022_05 not found, Downloading file...
File for 2022_06 not found, Downloading file...
File for 2022_07 not found, Downloading file...
File for 2022_08 not found, Downloading file...
File for 2022_09 not found, Downloading file...
File for 2022_10 not found, Downloading file...
File for 2022_11 not found, Downloading file...
File for 2022_12 not found, Downloading file...
File for 2023_01 not found, Downloading file...
File for 2023_02 not found, Downloading file...
File for 2023_03 not found, Downloading file...
File for 2023_03 not available
File for 2023_04 not found, Downloading file...
File for 2023_04 not available
File for 2023_05 not found, Downloading file...
File for 2023_05 not available
File for 2023_06 not found, Downloading file...
File for 2023_06 not available
File for 202

In [5]:
print(f"{len(rides)=}")

len(rides)=45635254


In [6]:
from src.data import processRawData

ts_data = processRawData(rides)

100%|██████████| 262/262 [00:01<00:00, 140.31it/s]


In [7]:
ts_data

,pickup_hour,rides,pickup_loc_id
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4
...,...,...,...
2666369,2023-02-28 20:00:00,0,199
2666370,2023-02-28 21:00:00,0,199
2666371,2023-02-28 22:00:00,0,199
2666372,2023-02-28 23:00:00,0,199


In [8]:
import hopsworks

project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/42132


In [9]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
FEATURE_GROUP_NAME = "ts_hourly_feat_group"
FEATURE_GROUP_VERSION = 1

In [11]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ["pickup_loc_id", "pickup_hour"],
    event_time = "pickup_hour"
)

In [12]:
feature_group.insert(ts_data, write_options={"wait_for_job": False} )

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/42132/fs/42052/fg/45529


Uploading Dataframe: 0.00% |          | Rows 0/2666374 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/42132/jobs/named/ts_hourly_feat_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f1e9cf03640>, None)